# Demo for VirtualCity
##### This is a demo of how to interact with VirtualCity. The demo will walk though how to start an environment and control it, how to prepare it to perform activities and finally how to perform activities in them.

## Setup
##### The first step is to start a communication with the simulator. Make sure before you run this notebook, that you have downloaded the correct version of the environment for your OS and your drivers and OS is updated to the latest version.

##### Install required packages

In [1]:
!pip install virtualcity
# !pip install python-socketio
# !pip install "python-socketio[client]"
# !pip install eventlet

  Using cached virtualcity-0.0.5-py3-none-any.whl.metadata (3.4 kB)
  Using cached ray-2.34.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached wandb-0.17.7-py3-none-win_amd64.whl.metadata (10 kB)
  Using cached stable_baselines3-2.3.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached matplotlib-3.9.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
Using cached virtualcity-0.0.5-py3-none-any.whl (2.7 kB)
Using cached ray-2.34.0-cp310-cp310-win_amd64.whl (24.7 MB)
Using cached stable_baselines3-2.3.2-py3-none-any.whl (182 kB)
Using cached wandb-0.17.7-py3-none-win_amd64.whl (6.5 MB)
Using cached GitPython-3.1.43-py3-none-any.whl (207 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
Using cached matplotlib-3.9.2-cp310-cp310-win_amd64.whl (7.8 MB)
Using cache

#### Communication server
##### This code is only run once to initialize the server. Run the following server code on a separate process (you can use a tmux server to run the code but avoid running it within the Jupyter Notebook file)

In [2]:
import socketio
import eventlet

sio = socketio.Server()
app = socketio.WSGIApp(sio)

clients = []

@sio.event
def connect(sid, environ):
    print('Client connected:', sid)
    clients.append(sid)

@sio.event
def disconnect(sid):
    print('Client disconnected:', sid)
    if sid in clients:
        clients.remove(sid)

@sio.event
def debug(sid, data):
    sio.emit('message', data)

@sio.event
def make(sid, data):
    sio.emit('make', data)

@sio.event
def reset(sid, data):
    sio.emit('reset', data)

@sio.event
def observation(sid, data):
    sio.emit('observation', data)

@sio.event
def set_action(sid, data):
    sio.emit('set_action', data)

@sio.event
def system_agent(sid, data):
    sio.emit('system_agent', data)

@sio.event
def render(sid, data):
    sio.emit('render', data)

if __name__ == '__main__':
    eventlet.wsgi.server(eventlet.listen(('0.0.0.0', 8000)), app)

OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

##### This code intializes the client communcation. It allows you to send commands to the server that you previously setup, which acts as a bridge between the executable file and the jupyter notebook. Run the following code to intialize the client communication with the environment

In [2]:
import socketio

sio = socketio.Client()

@sio.event
def connect():
    print("Connected to server")

@sio.event
def disconnect():
    print("Disconnected from server")

@sio.event
def message(data):
    print("Message from server:", data)

def make(data):
    sio.emit('make', data)

def reset(data):
    sio.emit('reset', data)

def observation(data):
    sio.emit('observation', data)

def set_action(data):
    sio.emit('set_action', data)

def system_agent(data):
    sio.emit('system_agent', data)

def render(data):
    sio.emit('render', data)

sio.connect('http://localhost:8081')

Connected to server


### Import packages
##### Before sending commands to the environment, you can import the utils package with provides additional interactive functionality with the environment

In [3]:

import virtualcity
import utils

ModuleNotFoundError: No module named 'virtualcity.communication'

## Sending commands to the environment
##### When you want to start a environment or send a instruction to an agent, etc. You will need to pack the configuration in a dictionary and use the emit function to send the command to the environment. If the command was succesfully executed in the environment, you will recieve a message: {success: True} and if it failed then you will recieve a message: {success: False}. 

### Make a environment
##### To start a environment, you need to provide the environment name that you want to initialize. You can select from various pre-built environments or use the "Procedural Generation" argument with a seed to create or recreate a previously generated environment on the fly.

In [4]:
"""
Args:
    environment (str): The name of the environment. Default is 'Demo'.
    num_env (list): The number of environments. Default is [1].
"""
data = {
    "environment": "KswainEscapeRoom3", 
    # "num_env": [1]
}

make(data)

### Reset the environment
##### To reset a environment, you can choose to provide a environment graph to reset it to a previosly saved state. If no graph is provided, then the environment is reset to its default state. 

In [5]:
"""
Args:
    env_index (list): The index of the environment. Default is [0].
    graph (dict): The environment graph in JSON. Default is None.
"""
import json

with open("graph.json", "r") as f:
    graph = json.load(f)

data = {
    "env_index": [0], 
    "graph": graph
}

reset(data)

FileNotFoundError: [Errno 2] No such file or directory: 'graph.json'

### Create a graph observation
##### You can create a environment graph that describes the entire environment in a JSON file. The graph contains object information, relations, states, etc.

In [6]:
"""
Args:
    type (str): The type of observation. Default is 'graph'.
    radius (list): The radius of the observation. Default is [-1].
"""
data = {
    "type": "graph", 
    # "radius": [-1]
}

observation(data)

### Set action for an agent
##### To send a action command to a agent, you can use provide a list containing the agent index, and a list describing the task which needs to be completed. The task list is structured in the following way: [action, object_1, relation, object_2]. To select a action with the appropriate object and relation, you can refer to the docs.

In [7]:
"""
Args:
    agent_index (list): The index of the agent. Default is [0].
    task (list): The task assigned to the agent. Default is [0, 0, 0, 0].
    coordinate (list): The coordinate of the agent. Default is [0, 0, 0].
"""
data = {
    "agent_index": [1],
    "task": [1, 1, 0, 0],
    # "coordinate": coordinate
}

set_action(data)

##### A more interactive way is to create list of actions in natural language, and run it with the sequence function from the optional utils package. Try to follow a similar syntax as the example provided below when running your own unique scripts.

In [8]:
import utils

# Sequence
action_list = [
    "agent_1 pickup object_1",
    "agent_2 walk to object_2",
    "agent_3 place object_4 on object_5",
    "agent_4 jump over object_6",
    "agent_5 examine object3 under object_7",
    "agent_6 run from object_8 to object_9",
    "agent_7 pick object_10 and place on object_11",
]

utils.sequence(action_list)

[{'agent_index': 1, 'task': [0, 1, 0, 0]},
 {'agent_index': 2, 'task': [2, 2, 0, 0]},
 {'agent_index': 3, 'task': [6, 3, 1, 4]},
 {'agent_index': 4, 'task': [0, 5, 0, 0]},
 {'agent_index': 5, 'task': [0, 6, 2, 7]},
 {'agent_index': 6, 'task': [3, 8, 0, 9]},
 {'agent_index': 7, 'task': [6, 10, 1, 11]}]

### Change system agent settings
##### You can also enable or disable system agents within the environment. Additional settings such as crowd density, vehicle denisity, parked cars, etc. can be changed via the API

In [9]:
"""
Args:
    crowd (bool): The crowd data. Default is True.
    traffic (bool): The traffic data. Default is True.
    visualize (bool): Visualize crowd and traffic. Default is False
"""
data = {
    "crowd": True, 
    "traffic": True,
    "visualize": False,
}
system_agent(data)

### Change rendering settings
##### You can also change the rendering settings of the environment in real time such as the camera, resolution, fps, etc. 

In [10]:
"""
Args:
    camera_index (list): The index of the camera. Default is [1].
    render_pipeline (str): Which renderer to use. Default is raytracing.
    image_width (list): The width of the image. Default is [1920].
    image_height (list): The height of the image. Default is [1080].
    fps (list): The frames per second. Default is [60].
"""
data = {
    "camera_index": [1],
    "render_pipeline": 'raytracing',
    "image_width": [1920],
    "image_height": [1080],
    "fps": [60],
}

render(data)